<a href="https://colab.research.google.com/github/Page0526/Pytorch-crash-course/blob/main/deep-neural-networks/nlp/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem
Your task is to reimplement Word2Vec algorithm, train the model on WikiText-103, and visualize the word embeddings using t-SNE. To make the visualization readable, you should not visualize all the words, about 100 words are enough for demonstration purposes.

Your submission is a Jupyter notebook file with the t-SNE visualization (I will not re-run your notebook to create the visualization).

# Example of unoptimized Word2Vec

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
corpus = """
From Wikipedia, the free encyclopedia

Example of a single industrial control loop; showing continuously modulated control of process flow.
A closed-loop controller or feedback controller is a control loop which incorporates feedback, in contrast to an open-loop controller or non-feedback controller. A closed-loop controller uses feedback to control states or outputs of a dynamical system. Its name comes from the information path in the system: process inputs (e.g., voltage applied to an electric motor) have an effect on the process outputs (e.g., speed or torque of the motor), which is measured with sensors and processed by the controller; the result (the control signal) is "fed back" as input to the process, closing the loop.[1]

In the case of linear feedback systems, a control loop including sensors, control algorithms, and actuators is arranged in an attempt to regulate a variable at a setpoint (SP). An everyday example is the cruise control on a road vehicle; where external influences such as hills would cause speed changes, and the driver has the ability to alter the desired set speed. The PID algorithm in the controller restores the actual speed to the desired speed in an optimum way, with minimal delay or overshoot, by controlling the power output of the vehicle's engine. Control systems that include some sensing of the results they are trying to achieve are making use of feedback and can adapt to varying circumstances to some extent. Open-loop control systems do not make use of feedback, and run only in pre-arranged ways.

Closed-loop controllers have the following advantages over open-loop controllers:

disturbance rejection (such as hills in the cruise control example above)
guaranteed performance even with model uncertainties, when the model structure does not match perfectly the real process and the model parameters are not exact
unstable processes can be stabilized
reduced sensitivity to parameter variations
improved reference tracking performance
improved rectification of random fluctuations[2]
In some systems, closed-loop and open-loop control are used simultaneously. In such systems, the open-loop control is termed feedforward and serves to further improve reference tracking performance.

A common closed-loop controller architecture is the PID controller.


A basic feedback loop
Open-loop and closed-loop
This section is an excerpt from Control loop § Open-loop and closed-loop.[edit]
Fundamentally, there are two types of control loop: open-loop control (feedforward), and closed-loop control (feedback).


An electromechanical timer, normally used for open-loop control based purely on a timing sequence, with no feedback from the process
In open-loop control, the control action from the controller is independent of the "process output" (or "controlled process variable"). A good example of this is a central heating boiler controlled only by a timer, so that heat is applied for a constant time, regardless of the temperature of the building. The control action is the switching on/off of the boiler, but the controlled variable should be the building temperature, but is not because this is open-loop control of the boiler, which does not give closed-loop control of the temperature.

In closed loop control, the control action from the controller is dependent on the process output. In the case of the boiler analogy this would include a thermostat to monitor the building temperature, and thereby feed back a signal to ensure the controller maintains the building at the temperature set on the thermostat. A closed loop controller therefore has a feedback loop which ensures the controller exerts a control action to give a process output the same as the "reference input" or "set point". For this reason, closed loop controllers are also called feedback controllers.[3]

The definition of a closed loop control system according to the British Standards Institution is "a control system possessing monitoring feedback, the deviation signal formed as a result of this feedback being used to control the action of a final control element in such a way as to tend to reduce the deviation to zero."[4]

Likewise; "A Feedback Control System is a system which tends to maintain a prescribed relationship of one system variable to another by comparing functions of these variables and using the difference as a means of control."[5]
Closed-loop transfer function
Main article: Closed-loop transfer function
The output of the system y(t) is fed back through a sensor measurement F to a comparison with the reference value r(t). The controller C then takes the error e (difference) between the reference and the output to change the inputs u to the system under control P. This is shown in the figure. This kind of controller is a closed-loop controller or feedback controller.

This is called a single-input-single-output (SISO) control system; MIMO (i.e., Multi-Input-Multi-Output) systems, with more than one input/output, are common. In such cases variables are represented through vectors instead of simple scalar values. For some distributed parameter systems the vectors may be infinite-dimensional (typically functions).

A simple feedback control loop
If we assume the controller C, the plant P, and the sensor F are linear and time-invariant (i.e., elements of their transfer function C(s), P(s), and F(s) do not depend on time), the systems above can be analysed using the Laplace transform on the variables. This gives the following relations:

PID feedback control
Main article: PID controller

A block diagram of a PID controller in a feedback loop, r(t) is the desired process value or "set point", and y(t) is the measured process value.
A proportional–integral–derivative controller (PID controller) is a control loop feedback mechanism control technique widely used in control systems.

A PID controller continuously calculates an error value e(t) as the difference between a desired setpoint and a measured process variable and applies a correction based on proportional, integral, and derivative terms. PID is an initialism for Proportional-Integral-Derivative, referring to the three terms operating on the error signal to produce a control signal.

The theoretical understanding and application dates from the 1920s, and they are implemented in nearly all analogue control systems; originally in mechanical controllers, and then using discrete electronics and later in industrial process computers. The PID controller is probably the most-used feedback control design.

If u(t) is the control signal sent to the system, y(t) is the measured output and r(t) is the desired output, and e(t) = r(t) − y(t) is the tracking error, a PID controller has the general form

However, in practice, a pure differentiator is neither physically realizable nor desirable[6] due to amplification of noise and resonant modes in the system. Therefore, a phase-lead compensator type approach or a differentiator with low-pass roll-off are used instead.
"""

# corpus = """A closed-loop controller or feedback controller is a control loop which incorporates feedback, in contrast to an open-loop controller or non-feedback controller. A closed-loop controller uses feedback to control states or outputs of a dynamical system. Its name comes from the information path in the system: process inputs (e.g., voltage applied to an electric motor) have an effect on the process outputs (e.g., speed or torque of the motor), which is measured with sensors and processed by the controller; the result (the control signal) is "fed back" as input to the process, closing the loop"""


In [ ]:
import re
import random

# generate training dataset
# for skip-gram negative sampling word2vec model
def create_dataset(corpus: str, window=2):
  # return dataset, V
  corpus = corpus.lower() # lower all characters
  # remove all non-alphabetic characters
  corpus = re.sub(r'[^a-z\s]', '', corpus).split()
  # print(corpus)
  vocab = sorted(set(corpus)) # save dictionary
  print(vocab)
  # V = len(vocab)
  word_to_idx = {w: i for i, w in enumerate(vocab)} # map each word to unique index
  idx_to_word = {i: w for i, w in enumerate(vocab)} # reverse dictionary

  data_pos = []
  # window mean range of context word respect to target word
  for i in range(window, len(corpus) - window - 1):
    context = corpus[i-window : i+window+1]
    del context[window] # delete target word from context word set
    # print(f"{context=}")
    target = corpus[i]
    # print(f"{target=}")
    data_pos += [(target, c, 1) for c in context]

  data_neg = []
  V = len(vocab)
  vocab_list = list(vocab)
  # print(len(vocab_list))
  for w in vocab_list:
    for _ in range(2 * window):
      idx = random.randint(0, V-1)
      neg_sample = vocab_list[idx]
      data_neg.append((w, neg_sample, 0))

  data = data_pos + data_neg
  random.shuffle(data)

  return vocab, data, word_to_idx, idx_to_word


In [ ]:
vocab, data, word_to_idx, idx_to_word = create_dataset(corpus=corpus)

In [ ]:
# V: vocab size
# d: dimension of the embedding
# w: V x d
# c: V x d
# Find w, c such that the loss is minimized
# Loss: BCE Loss

import matplotlib.pyplot as plt
import tqdm

def train(w, c, data, lr, n_epochs=10):
  lossf = nn.BCELoss()
  losses = []

  for ei in tqdm.trange(n_epochs):
    # print(f"Epoch {ei+1}/{n_epochs}")
    for di in data:
      i = word_to_idx[di[0]]
      j = word_to_idx[di[1]]
      wi = w[i, :]
      cj = c[j, :]
      wi = wi.reshape(-1, 1).requires_grad_()
      cj = cj.reshape(-1, 1).requires_grad_()
      pij = F.sigmoid(wi.T @ cj)
      # compute loss
      loss = lossf(pij, torch.FloatTensor([[di[2]]]))
      losses.append(loss.item())
      # backpropagate
      loss.backward()
      # udpate parameters
      with torch.no_grad():
        wi = wi - lr * wi.grad
        cj = cj - lr * cj.grad
      w[i, :] = wi.squeeze()
      c[j, :] = cj.squeeze()

  plt.plot(losses)

  return w, c

In [ ]:
d = 10
n_epochs = 100
lr = 1e-2

# initialize randomly target and context word
w = torch.randn(len(vocab), d)
c = torch.randn(len(vocab), d)
w, c = train(w=w, c=c, data=data, lr=lr, n_epochs=n_epochs)

In [ ]:
# visualize the word vectors with t-SNE
from sklearn.manifold import TSNE
import numpy as np

vectors = w + c
# create new tensor sharing same data but disconnected from computational graph
vectors = vectors.detach().numpy()

tsne = TSNE(n_components=2)
vectors = tsne.fit_transform(vectors)

plt.figure(figsize=(20, 20))
plt.scatter(vectors[:, 0], vectors[:, 1])
for i, word in enumerate(vocab):
  # thêm chú thích các từ trong plot
  plt.annotate(word, (vectors[i, 0], vectors[i, 1]))
plt.show()


# My solution
Modified from https://muhark.github.io/python/ml/nlp/2021/10/21/word2vec-from-scratch.html

## Prepare data

In [ ]:
!pip install datasets

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import torch
import datasets

dataset = datasets.load_dataset('Salesforce/wikitext','wikitext-103-v1')

'''
Preprocessing process:
1. lowercase everything
2. remove all symbol otherthan a-z@#
3. split on spaces
4. remove stopwords/ empty token
5. apply snowball stemmer to remainder (reduce word to their base (root) form. VD: cared, caring, careful -> care)
'''
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

ss = SnowballStemmer('english')
sw = stopwords.words('english')

In [ ]:
dataset

In [ ]:
# check data sample
for i in range(5):
  print(dataset['train'][i])
  print()

In [ ]:
import matplotlib.pyplot as plt

# get length of each text sample in dataset
train_length_texts = [len(sample['text'].split()) for sample in dataset['test']]

# plot the distribution length
plt.hist(train_length_texts, bins=50)
plt.xlabel('Text length(number of words)')
plt.ylabel('Frequency')
plt.title('Distribution of Text Lengths in Training set')
plt.show()

In [ ]:
testset = dataset['test']
testset

In [ ]:
def split_tokens(row):                             # STEP
    row['all_tokens'] = [ss.stem(i) for i in       # 5
                     re.split(r" +",               # 3
                     re.sub(r"[^a-z@# ]", "",      # 2
                            row['text'].lower())) # 1
                     if (i not in sw) and len(i)]  # 4
    return row

# determine vocab to create mapping
testset = testset.map(split_tokens)

In [ ]:
testset['all_tokens']

In [ ]:
print(testset)

In [ ]:
from collections import Counter

counts = Counter([i for s in testset['all_tokens'] for i in s])
counts = {k:v for k, v in counts.items() if v>10} # Filtering
vocab = list(counts.keys())
n_v = len(vocab)

id2tok = dict(enumerate(vocab))
tok2id = {token: id for id, token in id2tok.items()}

# Now correct tokens
def remove_rare_tokens(row):
    row['tokens'] = [t for t in row['all_tokens'] if t in vocab]
    return row

testset = testset.map(remove_rare_tokens)

In [ ]:
counts

In [ ]:
def windowizer(row, wsize=3):
    """
    Windowizer function for Word2Vec. Converts sentence to sliding-window
    pairs.
    """
    doc = row['tokens']
    wsize = 3
    out = []
    for i, wd in enumerate(doc):
        target = tok2id[wd]
        window = [i+j for j in
                  range(-wsize, wsize+1, 1)
                  if (i+j>=0) &
                     (i+j<len(doc)) &
                     (j!=0)]

        out+=[(target, tok2id[doc[w]]) for w in window]
    row['moving_window'] = out
    return row

testset = testset.map(windowizer)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class Word2VecDataset(Dataset):
  '''
  Takes a HuggingFace dataset as input, to be used for a Word2Vec Dataloader
  '''
  def __init__(self, dataset, vocab_size, wsize=3):
    self.dataset = dataset
    self.vocab_size = vocab_size
    self.data = [i for s in dataset['moving_window'] for i in s]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

In [ ]:
dataset.keys()

In [ ]:
testset

In [ ]:
BATCH_SIZE = 64
N_WORKERS = 2

dataloader = {}

dataloader = DataLoader(Word2VecDataset(testset, vocab_size=n_v), batch_size=BATCH_SIZE, shuffle=True, num_workers=N_WORKERS)

In [ ]:
dataloader

## Building Neural Network

### Manually one-hot-encoding

In [ ]:
from torch import nn

size = 10
input = 3

def one_hot_encode(input, size):
  vec = torch.zeros(size).float()
  vec[input] = 1.0
  return vec

ohe = one_hot_encode(input, size)
linear_layer = nn.Linear(size, 1, bias=False)

# set edge weight from 0 to 9 easy inference
with torch.no_grad():
  linear_layer.weight = nn.Parameter(torch.arange(10, dtype=torch.float).reshape(linear_layer.weight.shape))

print(linear_layer.weight)
# when pass ohe, retrieve weight corresponding to the input id
print(linear_layer(ohe))

In [ ]:
embedding_layer = nn.Embedding(size, 1)
with torch.no_grad():
  embedding_layer.weight=nn.Parameter(torch.arange(10, dtype=torch.float).reshape(embedding_layer.weight.shape))

  print(embedding_layer.weight)
  print(embedding_layer(torch.tensor(input)))

In [ ]:
from torch import nn
class Word2Vec(nn.Module):
  def __init__(self, vocab_size, embedding_size):
    super().__init__()
    self.embed = nn.Embedding(vocab_size, embedding_size)
    self.expand = nn.Linear(embedding_size, vocab_size, bias=False)

  def forward(self, input):
    # encoder input to lower-representation
    hidden = self.embed(input)
    # expand hidden layer to predictions
    logits = self.expand(hidden)
    return logits

In [ ]:
EMBED_SIZE = 100
# instantiate model
model = Word2Vec(n_v, EMBED_SIZE)

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
model.to(device)

LR = 3e-4
EPOCHS = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [ ]:
!pip show tqdm

In [ ]:
from tqdm import tqdm
progress_bar = tqdm(range(EPOCHS * len(dataloader)))
running_loss = []
for epoch in range(EPOCHS):
  epoch_loss=0
  for center, context in dataloader:
    # target word vs context words
    center, context = center.to(device), context.to(device)
    optimizer.zero_grad()
    logits = model(input=context)
    loss = loss_fn(logits, center)
    # loss is a tensor, loss.item() -> regular number float
    epoch_loss += loss.item()
    loss.backward()
    optimizer.step()
    progress_bar.update(1)

  epoch_loss/=len(dataloader)
  running_loss.append(epoch_loss)

In [ ]:
plt.plot(running_loss)

In [ ]:
tok2id

In [ ]:
wordvecs = model.expand.weight.cpu().detach().numpy() 
tokens = ['fun', 'upper', 'special', 'target']

from scipy.spatial import distance
import numpy as np

def get_distance_matrix(wordvecs, metric):
    dist_matrix = distance.squareform(distance.pdist(wordvecs, metric))
    return dist_matrix

def get_k_similar_words(word, dist_matrix, k=10):
    if word not in tok2id:
        print(f"Word '{word}' not in vocabulary.")
        return []
    
    idx = tok2id[word]
    dists = dist_matrix[idx]
    ind = np.argpartition(dists, k)[:k+1]
    ind = ind[np.argsort(dists[ind])][1:]
    out = [(i, id2tok[i], dists[i]) for i in ind]
    return out

dmat = get_distance_matrix(wordvecs, 'cosine')
for word in tokens:
    similar_words = get_k_similar_words(word, dmat)
    if similar_words:
        print(word, [t[1] for t in similar_words], "\n")

In [ ]:
wordvecs.shape

## Visualize wordvecs with T-SNE

In [ ]:
# visualize the word vectors with t-SNE
from sklearn.manifold import TSNE
import numpy as np

tsne = TSNE(n_components=2, random_state=42)
vectors = tsne.fit_transform(wordvecs[:100])

plt.figure(figsize=(20, 20))
plt.scatter(vectors[:, 0], vectors[:, 1])
for i, word in enumerate(vocab):
  # thêm chú thích các từ trong plot
  plt.annotate(word, (vectors[i, 0], vectors[i, 1]))
plt.show()